In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Phase 1 - ETL

Here i will load and transform the data i will be using;
Originally, that were datasets:
- anime_cleaned.csv; (cointains data about the animes)
- animelists_cleaned.csv; (contains the data about the users ratings of each anime)
- users_cleaned.csv (contains user data)

In order to ensure the experiment remains compatible with my system’s processing capacity, i needed to reduce the data volume, so i opted to:
- Remove animes in which the total number of ratings were bellow the median of all anime;
- Remove users who had whatch less than the median number of shows or more than the median plus the standart deviation, in order to handle outliers.

In this phase i also normalized the values.

## Step 1 - Function declaration

Here i will declare the functions that will be used for loading and transformation


In [3]:
# Paths for the raw files
datasets_folder_path: str = "./DataSets"
anime_file_name: str = "anime_cleaned.csv"
users_file_name: str = "users_cleaned.csv"
ratings_file_name: str = "animelists_cleaned.csv"


def generate_key_user_atributes() -> pd.DataFrame :
    """Generates a dataframe keeping only the user atributes that will be used 
    Args:
        path: String with the original dataset path
    Returns:
        users: pd.DataFrame containning usersnames, user_ids and number or completed shows
    """
    users = pd.read_csv(f"{datasets_folder_path}/{users_file_name}")
    users = users[["username","user_id","user_completed"]]
    
    return users


def generate_key_ratings_atributes() -> pd.DataFrame :
    """Generates a dataframe keeping only the user atributes that will be used 
    Args:
        path: String with the original dataset path
    Returns:
        ratings: pd.DataFrame containning usersnames, anime_ids and ratings"""
    ratings = pd.read_csv(f"{datasets_folder_path}/{ratings_file_name}")
    ratings = ratings[["username","anime_id","my_score"]]

    return ratings


def generate_key_anime_atributes() -> pd.DataFrame :
    """Generates a dataframe keeping only the user atributes that will be used 
    Args:
        path: String with the original dataset path
    Returns:
        ratings: pd.DataFrame containning anime_ids and the number of ratings"""
    animes = pd.read_csv(f"{datasets_folder_path}/{anime_file_name}")
    animes = animes [["anime_id","scored_by"]]

    return animes


def add_user_data_to_rating(ratings: pd.DataFrame, user_data: pd.DataFrame) -> pd.DataFrame:
    """Merges user data and rating data
    Args:
        ratings: pd.DataFrame with the ratings
        user_data: pd.DataFrame with the user data
    Returns:
        ratings: pd.DataFrame containing the merged DataFrames"""
    ratings = ratings.merge(user_data, on="username", how="inner")

    return ratings


def normalize_ratings(ratings: pd.DataFrame) -> pd.DataFrame:
    """Normalizes the column "my_score", by dividing all values by the biggest value
    Args:
        ratings: pd.Dataframe with the original data
    Returns:
        ratings: pd.DataFrame with an aditional column, called "normalized_score" """
    ratings["normalized_score"] = ratings["my_score"] / ratings["my_score"].max()

    return ratings


def remove_users_with_low_number_of_ratings(users: pd.DataFrame) -> pd.DataFrame:
    """Removes users who have rated less than the median number of anime or more than the median plus the standart deviation
    Args:
        users: pd.DataFrame with user data
    Returns:
        users: pd.Dataframe filtered, removing the users previously mentioned """
    std = users["user_completed"].std()
    median = users["user_completed"].median()
    upper_limit = median + std

    users = users.loc[
        (users["user_completed"] >= median) & 
        (users["user_completed"] <= upper_limit)
    ]

    return users


def remove_anime_with_few_ratings(animes: pd.DataFrame):
    """Removes anime with a number of ratings below the median
    Args:
        animes: pd.DataFrame with all anime data
    Returns:
        animes: pd.DataFrame with all anime data, except by those excluded above"""
    median = animes["scored_by"].median()
    animes = animes.loc[animes["scored_by"]> median]

    return animes

def drop_unused_info(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns that won't be used furthermore
    Args:
        df: pd.DataFrame
    Returns: 
        df: pd.DataFrame without the columns"""
    df = df.drop(columns=["username",
                          "my_score",
                          "user_completed",
                          "scored_by"])
    return df

## Step 2 - Defining a pipeline

Here i'll create a pipeline for loading, normalizing and joining the data by using the functions above

In [3]:
def load_normalize_and_join() -> pd.DataFrame:
    """Loads, normalizes and join the 3 main dataframes
    Returns:
        rating_data: pd.DataFrame containing anime_id, normalized_score and user_id"""

    users = generate_key_user_atributes()
    users = remove_users_with_low_number_of_ratings(users)

    ratings = generate_key_ratings_atributes()
    ratings = normalize_ratings(ratings)
    
    animes = generate_key_anime_atributes()
    animes = remove_anime_with_few_ratings(animes)

    rating_data = add_user_data_to_rating(ratings, users)
    # this step will remove all ratings from anime with less than the defined number of ratings
    rating_data = drop_unused_info(rating_data.merge(animes, on="anime_id", how="inner"))
    

    return rating_data

## Step 2.1 - runing the pipeline and storing the results
Run the cell below to execute the load, normalize and join pipeline, also, the following cell can be used to store the results in a .csv file, making it possible to only load the csv file instead of repeting the first steps.

In [4]:
rating_data = load_normalize_and_join()

In [5]:
rating_data.to_csv("./normalized_joined_data.csv", index=False)

In [6]:
rating_data = pd.read_csv("./normalized_joined_data.csv")

## Step 3 - Generating a Cosine Similarity Matrix

Here i'll create a matrix will all users, all animes, and the values of the normalized scores for each anime and user.

In [4]:
def create_user_anime_matrix(rating_data: pd.DataFrame) -> pd.DataFrame:
    """creates a matrix with all the values for users as the index and all the values for animes on the columns and fill the values with the ratings, for rated animes and with 0 in case there is not a rating
    Args: 
        rating_data: pd.DataFrame containing user_ids, anime_ids and normalized ratings
    Returns: 
        user_anime_matrix: pd.DataFrame containing a matrix with the source data"""
    user_anime_matrix = rating_data.pivot_table(
        index="user_id", columns="anime_id", values="normalized_score").fillna(0)
    
    return user_anime_matrix


def create_similarity_matrix(user_anime_matrix: pd.DataFrame) -> pd.DataFrame:
    """creats a similarity matrix by using an user-anime-rating matrix
    Args:
        user_anime_matrix: pd.DataFrame with a matrix of user/anime/rating
    Returns:
        similarity_matrix: pd.DataFrame with the level of similarity between all users"""
    user_similarity = cosine_similarity(user_anime_matrix)
    similarity_matrix = pd.DataFrame(user_similarity, index=user_anime_matrix.index, columns=user_anime_matrix.index)

    return similarity_matrix

## Step 4 - instantiate the matrix

Here i'll simple instantiate the matrixes on local variables, so they can be used later on

In [6]:
user_anime_matrix = create_user_anime_matrix(rating_data)
similarity_matrix = create_similarity_matrix(user_anime_matrix)

## Step 5 - Define the suggestion methods

Bellow, there is the definition of the functions used to get similar uses and, afterwards, an anime suggestion list, for an user_id

In [9]:
def get_similar_users(user_id: int, top_n: int=50) -> list:
    """Get a list with N most-similar users
    Args:
        user_id: int id belonged by the user
        top_n: int number of most-similar users returned
    Returns:
        similar: list of user_ids"""
    similar = similarity_matrix[user_id].sort_values(ascending=False)
    similar = similar.drop(user_id) 
    return similar.head(top_n)

def suggest_anime(user_id, top_n=10):
    """Get a dataframe with N anime_ids and average ratings for those animes
    Args:
        user_id: int id belonged by the user
        top_n: int number of anime suggestions
    Returns:
        non_watched_anime: pd.DataFrame containing anime_ids and average ratings"""
    similar_users = get_similar_users(user_id)
    
    similar_anime = user_anime_matrix.loc[similar_users.index]
    
    average_ratings = similar_anime.mean(axis=0)
    
    watched_anime = user_anime_matrix.loc[user_id]
    non_watched_anime = average_ratings[watched_anime == 0]

    anime = pd.read_csv(f"{datasets_folder_path}/{anime_file_name}")
    non_watched_anime = non_watched_anime.reset_index()
    non_watched_anime = non_watched_anime.merge(anime[["anime_id","title"]], on="anime_id", how="left")
    
    return non_watched_anime.sort_values(by=non_watched_anime.columns[1], ascending=False).head(top_n)

## Step 6 - Run the suggestion

Bellow, the method can be executed to return N suggestions for an user_id.

In [13]:
suggest_anime(user_id=120, top_n=20)

,anime_id,0,title
724,1735,0.770,Naruto: Shippuuden
672,1575,0.750,Code Geass: Hangyaku no Lelouch
1,6,0.718,Trigun
661,1559,0.676,Shijou Saikyou no Deshi Kenichi
29,71,0.666,Full Metal Panic!
155,270,0.658,Hellsing
450,889,0.654,Black Lagoon
738,1818,0.648,Claymore
902,2904,0.608,Code Geass: Hangyaku no Lelouch R2
773,2001,0.602,Tengen Toppa Gurren Lagann


# Bonus, adding aditional costum users to the matrix

An user can be added by appending it to the inicial user/anime/ratings dataframe, that can be done by gerating the user data, creating a new id and rating some animes in a csv, for instance:
- use the user id "123456789";
- find the id of some anime you like;
- rate with ratings from 0 to 1;

you can find the anime id using the 2 cells bellow

In [14]:
animes = pd.read_csv(f"{datasets_folder_path}/{anime_file_name}")
animes = animes.loc[animes["scored_by"]> (animes["scored_by"].median())]
animes = animes[["anime_id","title"]]

def get_animes(query: str) -> pd.DataFrame :
    """Get a list of anime_ids and titles
    Args:
        query: str with the name you look for
    Returns:
        results: pd.Dataframe with the filter results"""
    results = animes[animes["title"].str.contains(query, case=False, na=False)]

    return results

In [15]:
get_animes("re:zero")

,anime_id,title
4942,33142,Re:Zero kara Hajimeru Break Time
5137,31240,Re:Zero kara Hajimeru Isekai Seikatsu


# list exemple
- anime_id,normalized_score,user_id
- 31240, 1, 123456789,
- 29803, 1, 123456789,
- 269, 0.8, 123456789,
- 14813, 0.8, 123456789

add your list to the ratings list, bellow and re-run the similarity matrix (its important to note that the most anime there is in your list, the accuracy increases)

In [7]:
from io import StringIO

my_list = """anime_id,normalized_score,user_id
31240, 1, 123456789,
29803, 1, 123456789,
269, 0.8, 123456789,
14813, 0.8, 123456789"""
my_list = pd.read_csv(StringIO(my_list),index_col=False)

rating_data = pd.concat([rating_data, my_list], ignore_index=True)

user_anime_matrix = create_user_anime_matrix(rating_data)
similarity_matrix = create_similarity_matrix(user_anime_matrix)

In [10]:
suggest_anime(123456789)

,anime_id,0,title
1390,6547,0.254,Angel Beats!
1854,11757,0.246,Sword Art Online
761,1535,0.244,Death Note
2292,19815,0.226,No Game No Life
2904,31964,0.208,Boku no Hero Academia
2938,32281,0.206,Kimi no Na wa.
833,1735,0.206,Naruto: Shippuuden
2416,22199,0.196,Akame ga Kill!
2133,16498,0.190,Shingeki no Kyojin
2720,30276,0.186,One Punch Man
